# IEC Scrappeer

This scrapper mines data from www.icegate.gov.in for missing IEC Codes of Shipping Bill (Export).

Select the Excel file where :
    1. Location (Port Name) in Column A
    2. Shipping Bill Number in Column C
    3. Shipping Bill Date in Column B
    4. Log as NULL (Scrapped CHA Name will be store in this Column D)
    
Enter the Captcha text and click on Chrome Driver to pass the active instance as Chrome.

In [ ]:
# import xlwings as xw
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as cond
from selenium.webdriver.common.by import By
from tqdm import tqdm
from tqdm import tqdm_notebook
from selenium.common.exceptions import*
# from selenium.common.exceptions import NoSuchElementException
from tkinter.filedialog import askopenfilename
from tkinter import*
import win32com.client as win


# window to select the excel file containing data
root =Tk()
root.update()
path_file = filedialog.askopenfilename(initialdir = 'C://Users//data2//Desktop//IEC To Fetch', title = "Select your Excel file that contains input data:") 
root.destroy()


# load Workbook and Sheet
workbook= xw.Book(path_file)
# sheet = workbook["Sheet1"]  
sheet = workbook.sheets["Sheet1"]

# function to read Data from Excel Sheet
def readData(column_name, rownum):
    rownum1 = str(rownum) 
    cell = column_name+rownum1
    return sheet.range(cell).value
#     return sheet.cell(row = rownum, column = columnno).value

# function to write Data from Excel Sheet
def writeData(column_name, rownum, data):
    rownum1 = str(rownum)
    cell = column_name+ str(rownum)
    sheet.range(cell).value= data

#path to the chromedriver
driver = webdriver.Chrome(executable_path="chromedriver.exe") 
driver.set_page_load_timeout(10)


#url to the website data to be retrieve
driver.get("https://enquiry.icegate.gov.in/enquiryatices/sbTrack")

In [8]:
captchatxt = input("Enter the captcha: ")

Enter the captcha: tiE9A2


In [9]:
#send data to the textbox
count = 0
for x in tqdm_notebook(range(2, 205), desc = "Progress..."):
    log = readData("D", x)
    driver.set_page_load_timeout(5)
    if log is None:
        location = readData("A", x)
        sb_date = readData("B",x)
        sb_num = readData("C",x)
        try:
            sb_num = int(sb_num)
        except:
            pass
        #send data to form
        WebDriverWait(driver, 5).until(cond.presence_of_element_located((By.ID, "location"))).send_keys(location)
        driver.find_element_by_id('sbNO').send_keys(sb_num)
   

    #send date after remove calender
        sbDATE = driver.find_element_by_id('sbDATE')
        driver.execute_script("arguments[0].removeAttribute('readonly')", sbDATE)
        sbDATE.send_keys(sb_date)

        
    # send the captcha entered manually
        driver.find_element_by_id('captchaResp').send_keys(captchatxt)
        
        
    # click submit button
#     try:
        submit = WebDriverWait(driver,5).until(cond.element_to_be_clickable((By.ID,'SubB')))
        submit.click()
#         except :
#             try:
#                 wsh= win.Dispatch("WScript.Shell")
#                 wsh.AppActivate("Chrome")# select another application
#                 wsh.SendKeys('{F5}')
#             except:
#                 try:
#                     wsh= win.Dispatch("WScript.Shell")
#                     wsh.AppActivate("Chrome")# select another application
#                     wsh.SendKeys('{F5}')
#                     wsh.SendKeys('{F5}')
#                 except:
#                     try:
#                         wsh= win.Dispatch("WScript.Shell")
#                         wsh.AppActivate("Chrome")# select another application
#                         wsh.SendKeys('{F5}')
#                         wsh.SendKeys('{F5}')
#                     except:
#                         try:
#                             wsh= win.Dispatch("WScript.Shell")
#                             wsh.AppActivate("Chrome")# select another application
#                             wsh.SendKeys('{F5}')
#                             wsh.SendKeys('{F5}')
#                         except:
#                             wsh= win.Dispatch("WScript.Shell")
#                             wsh.AppActivate("Chrome")# select another application
#                             wsh.SendKeys('{F5}')
#                             wsh.SendKeys('{F5}')
                        
                        
        try:
            submit1 = WebDriverWait(driver, 5).until(cond.element_to_be_clickable((By.CLASS_NAME, "page_button")))
            submit1.click()
        except:
            try:
#                 submit = WebDriverWait(driver,5).until(cond.element_to_be_clickable((By.ID,'SubB')))
#                 submit.click()
#                 driver.set_page_load_timeout(3)
#                 driver.forward()
#                 driver.back()
                submit = WebDriverWait(driver,5).until(cond.element_to_be_clickable((By.ID,'SubB')))
                submit.click()
            except:
                home= None
                retry = 0
                driver.set_page_load_timeout(3)
                while home is None and retry<10:
                    try:
                        submit1 = WebDriverWait(driver, 1).until(cond.element_to_be_clickable((By.CLASS_NAME, "page_button")))
                        submit1.click()
                        home= WebDriverWait(driver, 1).until(cond.presence_of_element_located((By.CSS_SELECTOR, "#nav > li.current > a"))).text.split("&")
                    except:
                        wsh= win.Dispatch("WScript.Shell")
                        wsh.AppActivate("Chrome")
                        wsh.SendKeys('{F5}')
                        retry+=1
             

        
        try:
            temp_iec = WebDriverWait(driver, 1).until(cond.presence_of_element_located((By.CSS_SELECTOR, "#sbICES_Details > center > div > table > tbody > tr.tdText > td:nth-child(1)"))).text.split("&")
        except:
            writeData("D", x, 'Sorry :(')
            count = count+1
            driver.implicitly_wait(1)
            driver.back()
            continue
            
#         try:
#             temp_fob = driver.find_element_by_css_selector("#sbICES_Details > center > div > table > tbody > tr.tdText > td:nth-child(8)").text.split("&")
#            # temp_total =WebDriverWait(driver, 1).until(cond.presence_of_element_located((By.CSS_SELECTOR, "#beICES_Details > center > div > table > tbody > tr:nth-child(2) > td:nth-child(2)"))).text.split("&")
#         except:
#             pass
        iec = temp_iec[0]
#         fob = temp_fob[0]
        writeData("D", x, iec) 
#         writeData("E", x, fob)
        driver.set_page_load_timeout(10)
        driver.implicitly_wait(1) 
        driver.back()
    else:
        continue
# In[ ]:
driver.quit()
writeData("K", 11, "No record found for {} items".format(count))
print("No record found for {} items".format(count))
workbook.save(path_file)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



No record found for 0 items


In [10]:
driver.quit()